In [68]:
import pandas as pd
import numpy as np
import re
import time

import bs4 as bs4
import json

import glob
import tqdm

pd.set_option("max.columns", 131)

from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.preprocessing import MaxAbsScaler, StandardScaler
from scipy.sparse import csr_matrix

from lightgbm import LGBMClassifier

#https://strftime.org/
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/Users/mario/miniconda3/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['vstack', 'hstack']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [69]:
df = pd.read_csv("labels_curso - to_label_2.csv", index_col=0).dropna(subset=["y"])

In [70]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [71]:
df_limpo = pd.DataFrame(index=df.index)
df_limpo['title'] = df['watch-title']

## 1. Limpeza da data

In [72]:
clean_date = df['watch-time-text'].str.extract(r"(\d+) de ([a-z]+)\. de (\d+)")
clean_date[0] = clean_date[0].map(lambda x: "0"+x[0] if len(x) == 1 else x)
#clean_date[1] = clean_date[1].map(lambda x: x[0].upper()+x[1:])

mapa_meses = {"jan": "Jan",
              "fev": "Feb",
              "mar": "Mar", 
              "abr": "Apr", 
              "mai": "May", 
              "jun": "Jun",
              "jul": "Jul",
              "ago": "Aug", 
              "set": "Sep", 
              "out": "Oct", 
              "nov": "Nov",
              "dez": "Dec"}

clean_date[1] = clean_date[1].map(mapa_meses)

clean_date = clean_date.apply(lambda x: " ".join(x), axis=1)
clean_date.head()
df_limpo['date'] = pd.to_datetime(clean_date, format="%d %b %Y")

## 2. Limpeza de Views

In [73]:
views = df['watch-view-count'].str.extract(r"(\d+\.?\d*)", expand=False).str.replace(".", "").fillna(0).astype(int)
df_limpo['views'] = views

## 3. Features

In [74]:
features = pd.DataFrame(index=df_limpo.index)
y = df['y'].copy()

In [75]:
features['tempo_desde_pub'] = (pd.to_datetime("2019-12-03") - df_limpo['date']) / np.timedelta64(1, 'D')
features['views'] = df_limpo['views']
features['views_por_dia'] = features['views'] / features['tempo_desde_pub']
features = features.drop(['tempo_desde_pub'], axis=1)

In [76]:
features.head()

views  views_por_dia
0     28028      61.464912
394    1161      21.109091
393  141646     809.405714
392     325      21.666667
391      61       7.625000

In [77]:
mask_train = df_limpo['date'] < "2019-04-01"
mask_val = (df_limpo['date'] >= "2019-04-01")

Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((555, 2), (609, 2), (555,), (609,))

In [78]:
from sklearn.feature_extraction.text import TfidfVectorizer

title_train = df_limpo[mask_train]['title']
title_val = df_limpo[mask_val]['title']

title_vec = TfidfVectorizer(min_df=2, ngram_range=(1,3))
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)


In [79]:
title_bow_train.shape

(555, 1144)

In [80]:
from scipy.sparse import hstack, vstack

Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])

In [81]:
Xtrain_wtitle.shape, Xval_wtitle.shape

((555, 1146), (609, 1146))

# 4 RF

In [82]:
mdl_rf = RandomForestClassifier(n_estimators=1000, random_state=0, min_samples_leaf=1, class_weight="balanced", n_jobs=6)
mdl_rf.fit(Xtrain_wtitle, ytrain)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=6, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [83]:
p_rf = mdl_rf.predict_proba(Xval_wtitle)[:, 1]

In [84]:
average_precision_score(yval, p_rf), roc_auc_score(yval, p_rf)

(0.22228951304206077, 0.6914990859232175)

# 5 LGBM

In [104]:
params = [0.08265121231498246, 7, 1, 0.7251351011494334, 0.07547006552546137, 839, 2, 3]
lr = params[0]
max_depth = params[1]
min_child_samples = params[2]
subsample = params[3]
colsample_bytree = params[4]
n_estimators = params[5]

min_df = params[6]
ngram_range = (1, params[7])

title_vec = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range)
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)

Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])

mdl_lgbm = LGBMClassifier(learning_rate=lr, num_leaves=2 ** max_depth, max_depth=max_depth, 
                     min_child_samples=min_child_samples, subsample=subsample,
                     colsample_bytree=colsample_bytree, bagging_freq=1,n_estimators=n_estimators, random_state=0, 
                     class_weight="balanced", n_jobs=6)
mdl_lgbm.fit(Xtrain_wtitle, ytrain)

p_lgbm = mdl_lgbm.predict_proba(Xval_wtitle)[:, 1]


/Users/mario/miniconda3/lib/python3.7/site-packages/lightgbm/basic.py:497: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


In [105]:
average_precision_score(yval, p_lgbm), roc_auc_score(yval, p_lgbm)

(0.247808743128664, 0.6717874624049065)

# 7 Logistic Reg

In [87]:
from sklearn.pipeline import make_pipeline

In [88]:
Xtrain_wtitle2 = csr_matrix(Xtrain_wtitle.copy())
Xval_wtitle2 = csr_matrix(Xval_wtitle.copy())

#scaler = StandardScaler()
#scaler = MaxAbsScaler()

#Xtrain_wtitle2[:, :2] = scaler.fit_transform(Xtrain_wtitle2[:, :2].todense())
#Xval_wtitle2[:, :2] = scaler.transform(Xval_wtitle2[:, :2].todense())
#Xtrain_wtitle2 = scaler.fit_transform(Xtrain_wtitle2)
#Xval_wtitle2 = scaler.transform(Xval_wtitle2)

lr_pipeline = make_pipeline(MaxAbsScaler(), LogisticRegression(C=0.5, penalty='l2',n_jobs=6, random_state=0))
lr_pipeline.fit(Xtrain_wtitle2, ytrain)

/Users/mario/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/mario/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 6.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


Pipeline(memory=None,
     steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('logisticregression', LogisticRegression(C=0.5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn', n_jobs=6,
          penalty='l2', random_state=0, solver='warn', tol=0.0001,
          verbose=0, warm_start=False))])

In [89]:
p_lr = lr_pipeline.predict_proba(Xval_wtitle2)[:, 1]

In [90]:
average_precision_score(yval, p_lr), roc_auc_score(yval, p_lr)

(0.2130656734216374, 0.6834345697941853)

# 8 Ensemble

(0.22228951304206077, 0.6914990859232175) RF  
(0.23779186526938, 0.6883293035324645) LGBM  
(0.2124987281512838, 0.6808987438815827) LR  

(0.247808743128664, 0.6717874624049065) LGBM ngram 1,3

In [106]:
p = (p_lr + p_rf + p_lgbm)/3
average_precision_score(yval, p), roc_auc_score(yval, p)

(0.24346920623544438, 0.6936958188358789)

In [107]:
pd.DataFrame({"LR": p_lr, "RF": p_rf, "LGBM": p_lgbm}).corr()

LR        RF      LGBM
LR    1.000000  0.820536  0.460369
RF    0.820536  1.000000  0.486359
LGBM  0.460369  0.486359  1.000000

In [110]:
p = 0.5*p_rf + 0.5*p_lgbm
average_precision_score(yval, p), roc_auc_score(yval, p)

(0.24316089022038515, 0.6923394468361149)

(0.23117553771909904, 0.6964675355310491) - 0.5/0.5  
(0.23866391160240463, 0.6962906174441233) - 0.4/0.6  
(0.2449271153955049, 0.6967329126614378) - 0.3/0.7  
(0.24568903874837777, 0.6967329126614378) - 0.2/0.8  

(0.24567146005469367, 0.6897151618800496) - 0.3/0.7 - lgbm ngram 1,3
(0.24809974466463763, 0.690865129445067) - 0.4/0.6 

In [ ]:
# reduzir complexidade do vectorizer

# 9 Salvar modelos

In [111]:
import joblib as jb

In [112]:
jb.dump(mdl_lgbm, "lgbm_20200208.pkl.z")
jb.dump(mdl_rf, "random_forest_20200208.pkl.z")
#jb.dump(lr_pipeline, "logistic_reg_20200208.pkl.z")
jb.dump(title_vec, "title_vectorizer_20200208.pkl.z")

['title_vectorizer_20200208.pkl.z']